# Indigenous lands on which observatories are located

In [1]:
import pandas as pd
import folium
import numpy as np
import math
from astropy.coordinates import EarthLocation as EL
import astropy.units as u
import requests

In [22]:
sites = EL.get_site_names()
sites_csv = pd.read_csv('Observatory Locations - Sheet1.csv')

## Get map

In [17]:
folium_map = folium.Map(location=[43.6532, -79.3832],
                        zoom_start=3,
                        tiles="https://tiles.stadiamaps.com/tiles/outdoors/{z}/{x}/{y}{r}.png",attr='http://openstreetmap.org')

In [18]:
loc_track = set() # hack for unique sites
for site in sites:
    loc = EL.of_site(site).geodetic
    if loc not in loc_track:
        loc_track.add(loc)
        locstr = '{:.6f},{:.6f}'.format(loc[1].to(u.deg).value, loc[0].to(u.deg).value)
    #     print("https://native-land.ca/api/index.php?map=languages&position={:s}".format(locstr))

        response = requests.get("https://api.native-land.ca/index.php?maps=territories&position={:s}".format(locstr))
        infos = response.json()
        namelist = []
        for info in infos:
            territory_name = info['properties']['Name']
            territory_color = info['properties']['color']
            style_function = lambda x, fill_color = territory_color, color = territory_color: {'fillColor': fill_color, 'color': color}
            namelist.append(territory_name)
            folium.GeoJson(info['geometry'], name=territory_name, tooltip=territory_name, style_function=style_function).add_to(folium_map)

        namestr = ''
        c = 0
        for name in namelist:
            if c==(len(namelist)-1):
                namestr=namestr+"{:s}".format(name)
            else:
                namestr=namestr+"{:s},\n".format(name)
            c+=1
        popup_text = "Observatory Name: {}<br> Indigenous Land: {}"
        popup_text = popup_text.format(site,
                              namestr)
        popup = folium.Popup(popup_text, max_width=200)

        folium.Marker(location = [loc[1].to(u.deg).value, loc[0].to(u.deg).value], popup=popup).add_to(folium_map)


    #     ,radius=radius, color=color,popup=popup_text

In [19]:
folium_map